In [1]:
!git clone https://github.com/QCDHUB/SIDIS-Affinity

# Uncommment line below for Latex support in Colab
#!apt install texlive texlive-latex-extra texlive-fonts-recommended dvipng cm-super

import numpy as np
import pandas as pd
from pandas import read_excel 
from copy import deepcopy
from ipywidgets import *
import logging, os 
logging.disable(logging.WARNING) 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
print("tf.__version__", tf.__version__)
from functools import partial, reduce
from scipy.stats import percentileofscore
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import pylab as py
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
mpl.rcParams.update({"mathtext.fontset" : "custom", "mathtext.bf" : "STIXGeneral:bold", "mathtext.rm" : "STIXGeneral"})
%cd '/content/SIDIS-Affinity'
from plottools import plotEIC1

%matplotlib inline

# progress bar setup 
import time, sys
from IPython.display import clear_output

tf.__version__ 2.3.1


In [2]:
collinear_region_name = 'collinear'
current_region_name = 'current'
target_region_name = 'target'
TMD_region_name = 'TMD'
soft_region_name = 'soft'
collinear_lable_name = 'collinearaff'
target_lable_name = 'targetaff'
current_lable_name = 'currentaff'
TMD_lable_name = 'tmdaff'
soft_lable_name = 'softaff'

fname = './expdata/eic.xlsx'
data=pd.read_excel(fname,index_col=0, engine='openpyxl')

data.pT.max()
data.keys()
Q2b=data.Q2.unique()    
xb=data.x.unique()
test_features = deepcopy(data)
test_features = test_features.drop(columns=['W2', 'hadron', 'target'])

In [27]:
bins={}

for ix in range(len(xb)):
    for iQ2 in range(len(Q2b)):
        #print "iQ2=", len(Q2b)-iQ2-1, " ix= ", ix, ": ","Q2=="+str(Q2b[iQ2])+" and x=="+str(xb[ix])
        msg="Q2=="+str(Q2b[iQ2])+" and x=="+str(xb[ix])
        if data.query(msg).index.size != 0:
            bins[(len(Q2b)-iQ2-1,ix)]=msg
            
BINS = [(str(bins[b]),b) for b in sorted(bins)]


In [28]:
tmd_model_name = './models/final_%s' % TMD_region_name
tmd_model = tf.keras.models.load_model(tmd_model_name)
target_model_name = './models/final_%s' % target_region_name
target_model = tf.keras.models.load_model(target_model_name)
collinear_model_name = './models/final_%s' % collinear_region_name
collinear_model = tf.keras.models.load_model(collinear_model_name)
current_model_name = './models/final_%s' % current_region_name
current_model = tf.keras.models.load_model(current_model_name)
soft_model_name = './models/final_%s' % soft_region_name
soft_model = tf.keras.models.load_model(soft_model_name)

In [29]:
def update_progressz(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)
    
def above(dict,i,k):
      return (i-1,k) in dict.keys()

def below(dict,i,k):
      return (i+1,k) in dict.keys()

def left(dict,i,k):
      return (i,k-1) in dict.keys()

def right(dict,i,k):
      return (i,k+1) in dict.keys()

In [36]:
def plotEIC2(data, predictions, plotBin, hadron = 'pi+', affinity = 'tmdaff', ploty = 'qT', plotx = 'z', cmap_name = 'seismic_r', yscale = 'linear'):

    data['Q']=data['Q2']**0.5
    if 'had' not in data.keys() and 'hadron' in data.keys(): 
        data['had'] = data['hadron']
    
    if 'qT' not in data.keys():
        data['qT'] = data['pT']/data['z']
        
    # set affinity < 0 to 0 and affinity > 1 to 1
    predictions[predictions<0] = 0
    predictions[predictions>1] = 1
    data[affinity] = predictions  
        
    Q2b=data.Q2.unique()    
    xb=data.x.unique()
    zbins=data.z.unique()    
    
    bins={}
    
    for ix in range(len(xb)):
        for iQ2 in range(len(Q2b)):
            #print "iQ2=", len(Q2b)-iQ2-1, " ix= ", ix, ": ","Q2=="+str(Q2b[iQ2])+" and x=="+str(xb[ix])
            msg="Q2=="+str(Q2b[iQ2])+" and x=="+str(xb[ix])
            if data.query(msg).index.size != 0:
                bins[(len(Q2b)-iQ2-1,ix)]=msg

#     nrows,ncols=len(Q2b),len(xb)
    nrows,ncols=1,1
    fig = py.figure(figsize=(12,12))
#     gs = gridspec.GridSpec(nrows,ncols)
#     gs.update(wspace=0.,hspace=0,left=0.12, right=0.86,bottom=0.13,top=0.86)
    AX={}
    cmap = plt.get_cmap(cmap_name) # choose cmap
 
    
    # counter for progress bar
    barint = -1
    barint_max = 73
    
#     for k in sorted(bins):
    k = plotBin
    xbLabel = xb[k[1]]
    Q2bLabel = Q2b[8-k[0]]
    barint +=1 
    ir,ic=k
    #print k
    ax = py.subplot()
#     ax.set_xlim(0,8)
#     ax.set_ylim(0,1)
    #ax.set_xlim(0,data.qT.max())
    if plotx == 'z': 
        ax.set_xlim(0,1) # z is in [0,1]
#             ax2.set_xlim(0,1)
#             ax2.set_xlabel(r'$\mathbf{z_h}$', fontsize=70) 
    if ploty == 'pT': 
        if affinity.startswith('col'):
            max = 40
        elif affinity.startswith('tmd'):
            max = 10
        elif affinity.startswith('soft'):
            max = 10
        elif affinity.startswith('target'):
            max = 10
        else:
            max = 20
        ax.set_ylim(0,max) # pT is in [0,2]
#             ax2.set_ylim(0,max)
        ax.set_ylabel(r'$\mathbf{P_{hT} \; (GeV)}$', fontsize=30,x=-.5,y=.5) 
    if ploty == 'qT': 
        ax.set_ylim(0,15) #(0,data.qT.max())
#             ax2.set_ylim(0,15)
        ax.set_ylabel(r'$\mathbf{q_T \; \rm (GeV)}$', fontsize=30,x=-.5,y=.5)


    ax.set_yscale(yscale) # log or linear
    xticks = np.round(np.linspace(ax.get_xlim()[0],ax.get_xlim()[1],5),1)[1:4]
    yticks = np.round(np.linspace(ax.get_ylim()[0],ax.get_ylim()[1],5),1)[1:4]
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)

    d=data.query('%s and  had=="%s"'%(bins[k],hadron))

    for i in range(len(zbins)):

        msg='z > '+str(zbins[i]-zbins[i]/100)+' and z < '+ str(zbins[i]+zbins[i]/100)
        dd=d.query(msg)
        if dd.index.size==0: continue

        update_progressz(barint / (barint_max+2))
        plot = ax.scatter(dd[plotx],dd[ploty], s=500*dd[affinity]**0.2+10, c=dd[affinity], 
                              cmap=cmap, alpha=0.8,vmin=0,vmax=1,label='') 

        ax.tick_params(axis='x', which='major', labelsize=32, direction='in')
        ax.tick_params(axis='y', which='major', labelsize=35, direction='in')
    ax.annotate(r'$\mathbf{Q^2~({GeV}^2)}$', 
            xy=(-.3,.5),
            xycoords='axes fraction',
            size=40,
            rotation=90)

    ax.annotate(r'$\mathbf{x_{Bj}}$', 
            xy=(.5,-.3),
            xycoords='axes fraction',
            size=40)  
    if xbLabel<2e-3: msg=r'$%0.5f$'%xbLabel
    elif xbLabel<2e-2: msg=r'$%0.3f$'%xbLabel  
    else:msg=r'$%0.2f$'%xbLabel
#     msg=r'$%0.2f$'%xbLabel
    ax.text(0.5,-0.2,msg,transform=ax.transAxes,size=35,ha="center")
    ax.annotate('',xy=(0,-0.15),xycoords='axes fraction',xytext=(1, -0.15), 
                arrowprops=dict(arrowstyle="<->", color='k'))

    ax.text(-0.23,0.5,r'$%0.1f$'%Q2bLabel,
          transform=ax.transAxes,size=35,rotation=90,va="center")
    ax.annotate('',xy=(-.15,0),xycoords='axes fraction',xytext=(-0.15,1), 
                arrowprops=dict(arrowstyle="<->", color='k'))
    ax.annotate('', xy=(-0.09, 1), 
                xycoords='axes fraction', 
                xytext=(-0.09, 0),
                arrowprops=dict(arrowstyle="-|>, head_width=1, head_length=2", 
                color='k',lw=1.5))

    ax.annotate('', xy=(1,-0.06), 
                xycoords='axes fraction', 
                xytext=(0, -0.06),
                arrowprops=dict(arrowstyle="-|>, head_width=1, head_length=2", 
                color='k',lw=1.5))  
    ax.set_xlabel(r'$\mathbf{z_h}$', size=35) 
    
    cbar_ax = fig.add_axes([1., 0.2, 0.05, .5])
    cbar = fig.colorbar(plot,cax=cbar_ax)
    cbar.ax.tick_params(labelsize=30)
    time.sleep(2)
    update_progressz(barint / (barint_max+1))
    time.sleep(1)
    update_progressz(1)
#     outname = 'EIC_%s_vs_%s_%s_%s'%(ploty,plotx,hadron,affinity)
#     py.savefig('Figs/%s.pdf'%outname)  

In [37]:
# Create a slider widget for interactive Rmax selection
R0maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
R1maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
R2maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
regionDrop=widgets.Dropdown(
    options=[('tmd', TMD_lable_name), ('target', target_lable_name), 
             ('collinear', collinear_lable_name), ('current', current_lable_name),
             ('soft', soft_lable_name)],
    value=TMD_lable_name,
    description='Region:')
yAxisDrop=widgets.Dropdown(
    options=[('pT', 'pT'), ('qt', 'qT')],
    value='pT',
    description='y-axis:')
binDrop=widgets.Dropdown(
    options=BINS,
    value=(8,5),
    description='BIN:')
# plotBin = (8, 5)
# Define a function to predict tmdaff that corresponds the widgets Rmax values and return plot
def plot(R0max=R0maxSlider,R1max=R1maxSlider,R2max=R2maxSlider,region=regionDrop,yAxis=yAxisDrop,plotBin=binDrop):
       
    test_features['R0max'] = np.ones(len(test_features))*R0maxSlider.value
    test_features['R1max'] = np.ones(len(test_features))*R1maxSlider.value
    test_features['R2max'] = np.ones(len(test_features))*R2maxSlider.value

    if region == TMD_lable_name:
        predictions = tmd_model.predict(test_features).flatten()

    elif region == target_lable_name:
        predictions = target_model.predict(test_features).flatten()

    elif region == collinear_lable_name:
        predictions = collinear_model.predict(test_features).flatten()

    elif region == soft_lable_name:
        predictions = soft_model.predict(test_features).flatten()

    else:
        predictions = current_model.predict(test_features).flatten()

    return plotEIC2(data, predictions, plotBin, hadron = 'pi+', affinity = region, ploty = yAxis, plotx = 'z', cmap_name = 'Spectral', yscale = 'linear')

In [38]:
#Initiate user interaction 
execute = interactive(plot, {'manual': True},R0=R0maxSlider,R1max=R1maxSlider,R2max=R2maxSlider,region=regionDrop)
execute

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='R0max', max=1.0, min=0.1, s…